In [5]:
import openpyxl
import pandas as pd
import numpy as np
from numpy.linalg import norm

feature_names = ['Age',	'Sex',	'ALB',	'ALP',	'ALT',	'AST',	'BIL',	'CHE',	'CHOL',	'CREA',	'GGT',	'PROT']
noOfIterations = 100

def convertSign(value):
    if value == "positive":
        return 1
    elif value == "negative":
        return 0
    else:
        return -1

def generateSigns(feature_names):
    dataframe = openpyxl.load_workbook("outputHepatitisKernelShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_KernelShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_KernelShap = pd.DataFrame(dict)
     
    df_KernelShap.to_excel("signsHepatitisKernelShap.xlsx")
   
    #################################################################
    dataframe = openpyxl.load_workbook("outputHepatitisTreeShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_TreeShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3          
          
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_TreeShap = pd.DataFrame(dict)
    
    df_TreeShap.to_excel("signsHepatitisTreeShap.xlsx")    

    ############################################################################
    dataframe = openpyxl.load_workbook("outputHepatitisLIMELOOCV_100.xlsx")
    sheet = dataframe.active
    
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_LIME = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 10
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[5].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 5            
            
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_LIME = pd.DataFrame(dict)
   
    df_LIME.to_excel("signsHepatitisLIME.xlsx")
   

def stackRowsColumns(M, noOfRows, noOfColumns):
    pos = [0] * noOfRows
    for i in range (0, noOfRows):
        pos[i] = i
    
    for i in range(1, noOfRows):
        wsim = [0] * noOfRows
        for j in range(i, noOfRows):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
        aux = pos[max_index]
        pos[max_index] = pos[i]
        pos[i] = aux
    
    for i in range(1, noOfColumns):
        wsim = [0] * noOfColumns
        for j in range(i, noOfColumns):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
    return pos, M

def cosineSimilarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

def similarityCase(M, i):
    s = 0
    k = i - 1
    for j in range (1, k + 1):
        s = s + cosineSimilarity(M.loc[i], M.loc[i - j]) * (1 / j)
    return s

def similarityMatrix(M, noOfRows):
    s = 0
    for i in range(1, noOfRows):
        s = s + similarityCase(M, i)
    s = s / (noOfRows - 1)
    return s

def GlobalAlign(MSP, MS, noOfRows):
    if similarityMatrix(MSP, noOfRows) >  similarityMatrix(MS, noOfRows):
        max = similarityMatrix(MSP, noOfRows)
        min = similarityMatrix(MS, noOfRows)
    else:
        max = similarityMatrix(MS, noOfRows)
        min = similarityMatrix(MSP, noOfRows)
    return min / max
    #return similarityMatrix(MSP, noOfRows) / similarityMatrix(MS, noOfRows)


def createMSP(MP, MS, noOfRows):
    MS_copy = MS.copy(deep = True)
    MP_copy = MP.copy(deep = True)
    posS, MS_copy = stackRowsColumns(MS_copy, noOfRows, len(feature_names))
    posP, MP_copy = stackRowsColumns(MP_copy, noOfRows, len(feature_names))
    # MSP = pd.DataFrame(columns = feature_names)
    # for i in range(0, noOfRows):
    #     MSP.loc[posP[i]] = MS_copy.loc[posS[i]]
    return GlobalAlign(MP_copy, MS_copy, noOfRows)

generateSigns(feature_names)
df_KernelShap = pd.read_excel('signsHepatitisKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
df_TreeShap = pd.read_excel('signsHepatitisTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_LIME = pd.read_excel('signsHepatitisLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]

results = pd.DataFrame(columns = ['LL', 'LS', 'SL', 'SS'])
noOfInstances = df_KernelShap.shape[0]

for instance in range(0, noOfInstances):
    result = []
    data_Shap = []
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    
    data_LIME = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    
    MP = M_LIME
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_LIME
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    print(result)
    results.loc[instance] = result


results.to_excel("resultsSignsHepatitisMPMS_MinMax.xlsx")

[1.0, 0.8604345503033215, 0.8604345503033215, 1.0]
[1.0, 0.38842141736654834, 0.38842141736654834, 1.0]
[1.0, 0.6114198702168748, 0.6114198702168748, 1.0]
[1.0, 0.5983515992211774, 0.5983515992211774, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.6816745560482629, 0.6816745560482629, 1.0]
[1.0, 0.5053181046899975, 0.5053181046899975, 1.0]
[1.0, 0.5539798486168078, 0.5539798486168078, 1.0]
[1.0, 0.14324384380719357, 0.14324384380719357, 1.0]
[1.0, 0.3604318432842939, 0.3604318432842939, 1.0]
[1.0, 0.5929165624461461, 0.5929165624461461, 1.0]
[1.0, 0.4338618955261563, 0.4338618955261563, 1.0]
[1.0, 0.07338303470559389, 0.07338303470559389, 1.0]
[1.0, 0.5417983478523798, 0.5417983478523798, 1.0]
[1.0, 0.530319996421335, 0.530319996421335, 1.0]
[1.0, 0.5072368765151676, 0.5072368765151676, 1.0]
[1.0, 0.35966026244639376, 0.35966026244639376, 1.0]
[1.0, 0.49095957414158786, 0.49095957414158786, 1.0]
[1.0, 0.6778074411929201, 0.6778074411929201, 1.0]
[1.0, 0.5097751571645787, 0.5097751571645787, 1.0]
[1.0, 0.6772293775665494, 0.6772293775665494, 1.0]
[1.0, 0.3329910147464956, 0.3329910147464956, 1.0]
[1.0, 0.5277148882368098, 0.5277148882368098, 1.0]
[1.0, 0.9165052596304635, 0.9165052596304635, 1.0]
[1.0

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.2947039467313808, 0.2947039467313808, 1.0]
[1.0, 0.5771454256785095, 0.5771454256785095, 1.0]
[1.0, 0.6645834508158981, 0.6645834508158981, 1.0]
[1.0, 0.6027430218802311, 0.6027430218802311, 1.0]


...

[1.0, nan, nan, nan]
[1.0, 0.48909488541884005, 0.48909488541884005, 1.0]
[1.0, 0.3564634563247883, 0.3564634563247883, 1.0]
[1.0, 0.3436705396619442, 0.3436705396619442, 1.0]
[1.0, 0.5433275439695161, 0.5433275439695161, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.7081692763825778, 0.7081692763825778, 1.0]
[1.0, 0.3364759046140786, 0.3364759046140786, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_13160\3787471243.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.4398880029261875, 0.4398880029261875, 1.0]
[1.0, 0.6869672010303146, 0.6869672010303146, 1.0]
[1.0, 0.35918880907397455, 0.35918880907397455, 1.0]
[1.0, 0.5688516501862015, 0.5688516501862015, 1.0]
[1.0, 0.5302266039328298, 0.5302266039328298, 1.0]
[1.0, 0.6446012275431819, 0.6446012275431819, 1.0]
[1.0, 0.9128136811498918, 0.9128136811498918, 1.0]
[1.0, 0.6064834805456936, 0.6064834805456936, 1.0]
[1.0, 0.3442097855513544, 0.3442097855513544, 1.0]
[1.0, 0.6819658252614316, 0.6819658252614316, 1.0]
[1.0, 0.8012956443286261, 0.8012956443286261, 1.0]
[1.0, 0.8326465171818447, 0.8326465171818447, 1.0]
[1.0, 0.7739228556051359, 0.7739228556051359, 1.0]
[1.0, 0.6112095552986008, 0.6112095552986008, 1.0]
[1.0, 0.5957845700568226, 0.5957845700568226, 1.0]
[1.0, 0.7596568874139052, 0.7596568874139052, 1.0]
[1.0, 0.947849161028155, 0.947849161028155, 1.0]
[1.0, 0.9320294995693887, 0.9320294995693887, 1.0]
[1.0, 0.8852941059688902, 0.8852941059688902, 1.0]
[1.0, 0.79